In [2]:
import tensorflow as tf
import numpy as np

In [3]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '0', ' ']

In [4]:
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

In [5]:
seq_data = ['word', 'world', 'treatment', 'powerful', 'potential', 'google', 'south korea', 'the united states', 'natural language processing']

In [6]:
def get_max_length(seq_data):
    max_length = 0
    for seq in seq_data:
        if max_length < len(seq) - 1:
            max_length = len(seq) - 1
    return max_length

In [7]:
def get_seq_length(seq_data):
    seq_len = []
    for seq in seq_data:
        seq_len.append(len(seq)-1)
    return seq_len

In [8]:
seq_length_data = get_seq_length(seq_data)

In [9]:
seq_length_data

[3, 4, 8, 7, 8, 5, 10, 16, 26]

In [10]:
max_len = get_max_length(seq_data)

In [11]:
max_len

26

In [12]:
def give_zero_padding(seq_data, max_len):
    padded_data = []
    for seq in seq_data:
        temp = seq[:-1]
        for _ in range(len(seq), max_len+1):
            temp += '0'
        temp += seq[-1]
        padded_data.append(temp)
    return padded_data

In [13]:
seq_data = give_zero_padding(seq_data, max_len)

In [15]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
        
    return input_batch, target_batch

In [16]:
input_batch, target_batch = make_batch(seq_data)

In [17]:
learning_rate = 0.001
n_hidden = 128
total_epoch = 200
n_step = max_len
n_input = n_class = dic_len
batch_size = len(seq_data)

In [28]:
tf.reset_default_graph()
X = tf.placeholder(dtype=tf.float32, shape=[None, n_step, n_input])
Y = tf.placeholder(dtype=tf.int32, shape=[None])
seq_len = tf.placeholder(dtype=tf.int32, shape=[None])

In [29]:
W = tf.Variable(tf.truncated_normal([n_hidden, n_class]))
b = tf.Variable(tf.truncated_normal([n_class]))

In [30]:
cell = tf.nn.rnn_cell.GRUCell(n_hidden)
cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=0.5)

In [31]:
_, states = tf.nn.dynamic_rnn(cell, X, sequence_length=seq_len, dtype=tf.float32)
#outputs: TensorShape([batch_size, time_steps, hidden_size])

In [32]:
#idx = tf.range(tf.shape(outputs)[0])*tf.shape(outputs)[1] + (seq_len - 1)
#idx = tf.cast(tf.reduce_sum(tf.one_hot(idx, tf.shape(outputs)[0]*tf.shape(outputs)[1]), 0), tf.int32)
#outputs = tf.dynamic_partition(tf.reshape(outputs, [-1, n_hidden]), idx, 2)
#last_outputs = outputs[1]

In [33]:
#last_outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), idx)
#tf.reshape(outputs, [-1, n_hidden]): TensorShape([batch_size*time_steps, hidden_size])

In [34]:
logits = tf.matmul(states, W) + b
preds = tf.cast(tf.argmax(logits, 1), tf.int32)
correct = tf.equal(preds, Y)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [35]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=Y))

In [36]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [37]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch, seq_len: seq_length_data})
    if epoch % 20 == 19:
        print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.6f}'.format(loss))
    
print('optimization finished!')

Epoch: 0020 cost = 1.760996
Epoch: 0040 cost = 0.702635
Epoch: 0060 cost = 0.228974
Epoch: 0080 cost = 0.031601
Epoch: 0100 cost = 0.064658
Epoch: 0120 cost = 0.016007
Epoch: 0140 cost = 0.023214
Epoch: 0160 cost = 0.013288
Epoch: 0180 cost = 0.004314
Epoch: 0200 cost = 0.002514
optimization finished!


In [38]:
input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val = sess.run([preds, accuracy], feed_dict={X: input_batch, Y: target_batch, seq_len: seq_length_data})
for i, val in enumerate(seq_data):
    predict_words = seq_data[i][:seq_length_data[i]] + char_arr[predict[i]]
    print('Input: ', seq_data[i][:seq_length_data[i]])
    print('Prediction: ', predict_words, '\n')

Input:  wor
Prediction:  word 

Input:  worl
Prediction:  world 

Input:  treatmen
Prediction:  treatment 

Input:  powerfu
Prediction:  powerful 

Input:  potentia
Prediction:  potential 

Input:  googl
Prediction:  google 

Input:  south kore
Prediction:  south korea 

Input:  the united state
Prediction:  the united states 

Input:  natural language processin
Prediction:  natural language processing 

